In [ ]:
from keybert import KeyBERT
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from concurrent.futures import ThreadPoolExecutor, as_completed
kw_model = KeyBERT('all-mpnet-base-v2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


import error: No module named 'triton'


In [29]:
def extract_keywords(description, stop_words=stopwords.words('english')):
    return id, kw_model.extract_keywords(description, keyphrase_ngram_range=(1, 1), stop_words=stop_words)
def process_batch(batch_des, batch_id_image, batch_id, timeout=10):
    keywords = []
    with ThreadPoolExecutor() as executor:
        future_to_description = {executor.submit(extract_keywords, batch_des[i]): i for i in range(len(batch_des))}  
        for future in as_completed(future_to_description, timeout=timeout):
            try:
                keyword_tuples = future.result()[1]  # Extracts the full keyword list (with scores)
                keyword_list = [word for word, _ in keyword_tuples]  # Extract only the words

                keywords.append({
                    "id": batch_id[future_to_description[future]],
                    "id_image": batch_id_image[future_to_description[future]],
                    "keywords": keyword_list
                })
                # print(f"id_image {batch_id_image[future_to_description[future]]} id {batch_id[future_to_description[future]]} processed , reuslt: {keyword_list} ")
            except Exception as e:
                print(f"Error processing description: {e}")
    return keywords

In [ ]:
df = pd.read_csv('merge.csv')
descriptions = df['Image description'].tolist()
ids_image = df['Image'].tolist()
id = np.arange(0, len(descriptions))
batch_size = 10
all_keywords = []
for i in range(0, len(descriptions), batch_size):
    batch_des = df['Image description'][i:i+batch_size].tolist()
    batch_id_image = df['Image'][i:i+batch_size].tolist()
    id = np.arange(i, i+batch_size)
    all_keywords.extend(process_batch(batch_des,batch_id_image, id))
    print(f"Processed {i+batch_size} descriptions")

In [ ]:
# sort all keywords dict by id
all_keywords = sorted(all_keywords, key=lambda x: x['id'])
pd.to_csv('keywords.csv', all_keywords)
